In [6]:
import app 
import importlib
importlib.reload(app)

nlp = app.main()
type(nlp)

%load_ext autoreload
%autoreload 2


----Printing patterns, should be empty at this point----
[]


In [8]:
nlp.get_pipe("entity_ruler").patterns[1]

{'label': 'NEG', 'pattern': [{'LOWER': 'negativt'}]}

In [9]:
doc = nlp("Kolera, är inte en hemsk sjukdom")
print([(ent.text, ent.label_, ent.ent_id_) for ent in doc.ents])

[('inte', 'NEG', '')]


In [7]:
# Analyse the pipeline and store the analysis under 'pipe_analysis'
pipe_analysis = nlp.analyze_pipes(pretty=True)
# print(nlp.pipe_names)  # see all components in the pipeline


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   transformer       doc._.trf_data                                      False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

In [6]:
from visualization.spacy_tabulate import spacy_tabulate
spacy_tabulate(doc)

+------+-------+---------+---------+-------------+--------+--------------+
| NE   | IOB   | text    | lemma   | POS         | dep    | right edge   |
|------+-------+---------+---------+-------------+--------+--------------|
|      | O     | Kolera  | Kolera  | noun        | nsubj  | ,            |
|      | O     | ,       | ,       | punctuation | punct  | ,            |
|      | O     | är      | vara    | auxiliary   | cop    | är           |
| NEG  | B     | inte    | inte    | particle    | advmod | inte         |
|      | O     | en      | man     | determiner  | det    | en           |
|      | O     | hemsk   | hemsk   | adjective   | amod   | hemsk        |
|      | O     | sjukdom | sjukdom | noun        | ROOT   | sjukdom      |
+------+-------+---------+---------+-------------+--------+--------------+


In [8]:
from visualization.spacy_dispacy import spacy_displacy
spacy_displacy(doc,"ent",True) 

# Run & Evaluate model on corpus data 

In [1]:
import importlib
from make_corpus import convert_raw_to_spacy
# importlib.reload(make_corpus)
# Somehow this works fine when running in shell terminal but not in the subprocess where it can't find the file
convert_raw_to_spacy()

/home/callebalik/clinical_NLP_SE
python -m spacy convert /home/callebalik/clinical_NLP_SE/data/raw/corpus/conll2003_NE_built_in_layer/ /home/callebalik/clinical_NLP_SE/data/processed/corpus/conll2003_NE_built_in_layer/ --converter conll --n-sents 100
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents):
/home/callebalik/clinical_NLP_SE/data/processed/corpus/conll2003_NE_built_in_layer/chart1.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents):
/home/callebalik/clinical_NLP_SE/data/processed/corpus/conll2003_NE_built_in_layer/chart2.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents):
/home/callebalik/clinical_NLP_SE/data/processed/corpus/conll2003_NE_built_in_layer/chart3.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents):
/home/callebalik/clinical_NLP_SE/data/processed/corpus/conll2003_NE_built_in_layer/chart4.spacy
0


In [11]:
# Evaluate model
from evaluate import save_model
save_model(nlp,"test2")

In [15]:
from evaluate import evaluate_model

evaluate_model("test2")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Using CPU

================================== Results ==================================

TOK      -    
TAG      0.00 
POS      -    
LEMMA    -    
UAS      -    
LAS      -    
NER P    66.67
NER R    36.76
NER F    47.39
SENT P   48.78
SENT R   70.42
SENT F   57.64
SPEED    117  


=============================== NER (per type) ===============================

           P        R        F
TME     0.00     0.00     0.00
NEG   100.00   100.00   100.00
FND     0.00     0.00     0.00
DIS     0.00     0.00     0.00
SYM     0.00     0.00     0.00
ORG     0.00     0.00     0.00
PRS     0.00     0.00     0.00
LOC     0.00     0.00     0.00

✔ Generated 25 parses as HT

In [16]:
# Display model prediction as html 
import IPython
# IPython.display.HTML(filename=Path(repo_path / 'models/parses.html'))
model_name = "test2"
IPython.display.HTML(filename=f"../models/{model_name}/entities.html")

In [49]:
# Here we can get the original annotation to compare 
from spacy.tokens import DocBin
from spacy import displacy

doc_bin = DocBin().from_disk("/home/callebalik/clinical_NLP_SE/data/processed/corpus/conll2003_NE_built_in_layer/chart1.spacy")
docs = list(doc_bin.get_docs(vocab=nlp.vocab))

print(docs)
for d in docs:
    for c in d:
        print(c.ent_type_)


style = "ent"

displacy.render(docs, style=style, jupyter=True)


[Kontaktorsak : Dyspné Tidigare sjukdomar : Insulin och tablettbehandlad DM2 . Tablettbehandlad hypertoni . Höga blodfetter . Socialt : Ensamboende , hemtjänst * 1 . Aktuellt : Insjuknat för tre dagar sedan med andnöd vid gång . Tillkomst av hosta under gårdagen . Ej noterat feber , men ej eller mätt tempen . Idag nedsatt AT , svårt att gå . Tagit sina mediciner som vanligt . Sökt VC som hänvisat hit pga destruering . Inte haft några utslag . Ej kontakt med någon annan som varit sjuk . Ej varit ute och rest . Ingen smärta vid miktion , buksmärta eller ÖNH symptom . Idag sämre , kan ej mobilisera sig i hemmet . Inkommer på remiss . Inga kräkningar eller GI symptom . Status : AT : Gott , opåverkad i vila , ingen samtalsdyspné . Cor : Normofrekvent regelbunden rytm , inga hörbara bi eller blåsljud . Pulm : I stort vesikulära antingsljud bilat , dämpat basalt höger . Inget förlängt expirium . Buk : Överallt muk och oöm . Ingen lokal peritonit . Lokalstatus : Underben smala och gracila , in